# Objetivo de la prueba

En este caso, se quiere pronosticar la demanda de un producto para una semana
determinada, en una tienda especial. El conjunto de datos que recibe consta de 8 semanas
de transacciones de ventas en México.

Cada semana, hay camiones de reparto que entregan productos a los vendedores. Cada
transacción consiste en ventas y devoluciones. Las devoluciones son los productos que no
se han vendido y caducado.

La demanda de un producto en una semana determinada se define como las ventas de esta
semana restado por el retorno de la próxima semana.

## Requisitos

● Intentar desarrollar la prueba en Python / R / SQL, o el programa que se le facilite.

● Comparte el documento con las respuestas, incluye una breve explicación de tus resultados.

● Comparta su código para revisarlo en detalle.

## Entregables

1. Justifica la elección del algoritmo que utilizaste.

2. Graficar una serie de tiempo para una determinada combinación producto-cliente-agencia.

3. Agregue varias métricas de su elección para mostrar la solidez del algoritmo.

4. Definir una muestra de clientes que represente a una agencia específica (Especifique cuál agencia y cuáles fueron sus criterios de selección y metodología aplicada).

5. Prediga la semana 9 para los 3 productos más vendidos en la muestra de su cliente.

6. Haz un dibujo de diagrama de flujo que mapee todos los pasos de tu algoritmo usado.

## Cosas a tener en cuenta

● Puede haber productos en el conjunto de prueba que no existen en el conjunto
de entrenamiento. Este es el comportamiento de los datos de inventario, ya que
hay nuevos productos que se venden todo el tiempo. Tu modelo debería ser
capaz de acomodar esto.

● Hay Cliente_ID duplicados en cliente_tabla, lo que significa un Cliente_ID. Puede
tener múltiples NombreCliente que son muy similares. Esto se debe al
NombreCliente ser ruidoso y no estandarizado en los datos sin procesar, por lo
que depende de usted decidir cómo limpiar y utilizar esta información.

● La demanda ajustada (Demanda_uni_equil) siempre es >= 0 ya que la demanda
debería ser un 0 o un valor positivo. La razón que Venta_uni_hoy -
Dev_uni_proxima a veces tiene valores negativos es que el los registros de
devoluciones a veces se prolongan durante algunas semanas.

## Descripciones de archivos

● cliente_tabla.csv: nombres de clientes (se pueden unir con tren/prueba en
Cliente_ID)

● producto_tabla.csv: nombres de productos (se pueden unir con tren/prueba en
Producto_ID)

● estados_state.csv: ciudad y estado (se puede unir con tren/prueba en Agencia_ID) Campos de información

● Semana — Número de semana (de jueves a miércoles)

● Agencia_ID: ID del depósito de ventas

● Canal_ID: ID del canal de ventas

● Ruta_SAK: ID de ruta (Varias rutas = Depósito de ventas)

● Cliente_ID: identificación del cliente

● NombreCliente — Nombre del cliente

● Producto_ID — Identificación del producto

● NombreProducto — Nombre del producto

● Venta_uni_hoy: unidad de ventas de esta semana (entero)

● Venta_hoy — Ventas esta semana (unidad: pesos)

● Dev_uni_proxima: devuelve la unidad la próxima semana (entero)

● Dev_proxima — Vuelve la próxima semana (unidad: pesos)

● Demanda_uni_equil — Demanda ajustada (entero) (Este es el objetivo que predecir)

# Preparación de los datos

## Limpiar tabla de clientes
Hay Cliente_ID duplicados en cliente_tabla, lo que significa un Cliente_ID. Puede
tener múltiples NombreCliente que son muy similares. 

Esto se debe al NombreCliente ser ruidoso y no estandarizado en los datos sin procesar, por lo
que depende de usted decidir cómo limpiar y utilizar esta información.

In [ ]:
# Importar libreria para leer dataset
import pandas as pd

In [ ]:
#pip install google.colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Incluye el link al CSV de tu Google Drive donde se ha guardado el Data Set
ad_df = pd.read_csv('/content/drive/My Drive/advertising.csv')

In [ ]:
ad_df.describe()

In [ ]:
# cargar dataset de cliente mediante url de repositorio publico en github para realizar proceso de data cleaning
df_duplicate = pd.read_csv("https://media.githubusercontent.com/media/EnriqueEscalante91/Caso-MELI/master/cliente_tabla.csv")
df_duplicate.shape

In [ ]:
# Mostrar los primeros registros del DataFrame
print(df_duplicate.head()) 

In [4]:
#Validar si existen registros duplicados
duplicates = df_duplicate[df_duplicate.duplicated('Cliente_ID')]
# Mostrar los registros duplicados
print("Registros duplicados:")
print(df_duplicate)

Registros duplicados:
        Cliente_ID                            NombreCliente
0                0                               SIN NOMBRE
1                1                         OXXO XINANTECATL
2                2                               SIN NOMBRE
3                3                                EL MORENO
4                4  SDN SER  DE ALIM  CUERPO SA CIA  DE INT
...            ...                                      ...
935357    11011586                               OXXO PETEN
935358    11693264                     SUPER ABARROTES MARY
935359    19988629                          NO IDENTIFICADO
935360    99999999                          NO IDENTIFICADO
935361  2015152015                          NO IDENTIFICADO

[935362 rows x 2 columns]


In [5]:
# Eliminar los registros duplicados en la columna 'Cliente_ID'
df_cleaned = df_duplicate.drop_duplicates(subset='Cliente_ID', keep='first', inplace=True)

In [6]:
# Mostrar el DataFrame resultante
print("DataFrame sin duplicados:")
print(df_cleaned)

DataFrame sin duplicados:
None


In [7]:
# Guardar el DataFrame limpio en un nuevo archivo CSV
#csv_filename = "cliente_tabla_sin_duplicados.csv"
#df.to_csv(csv_filename, index=False)

# Desarrollo del modelo

LSTM (Long Short-Term Memory) en TensorFlow y Keras es altamente recomendado para pronósticos de series temporales debido a su capacidad para capturar patrones a largo plazo y dependencias temporales. Las capas LSTM manejan secuencias variables, conservan información crucial a lo largo del tiempo y evitan problemas de desvanecimiento del gradiente. Ofrecen flexibilidad para configurar hiperparámetros y se integran en modelos Sequential. Estas características los hacen adecuados para capturar relaciones complejas en datos secuenciales y facilitan el pronóstico preciso en problemas de series temporales como la predicción de la demanda basada en datos históricos.


In [8]:
# Importar libreria pandas en caso de no haber ejecutado antes
#import pandas as pd
# Importar Tensorflow para nuestro modelo de machine learning
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
#importar libreria para conectar con base de datos SQL server
from sqlalchemy import create_engine

## Entregable 1
El algoritmo LSTM (Long Short-Term Memory) en un modelo construido con TensorFlow y Keras tiene capacidad para tratar con secuencias de datos, como series temporales, y capturar patrones a largo plazo en los datos. Aquí hay algunas razones para considerar el uso de un modelo LSTM:

Captura de Dependencias Temporales: En problemas de pronóstico de series temporales, las dependencias temporales son cruciales. Las capas LSTM están diseñadas para capturar y recordar patrones de dependencia a lo largo del tiempo, lo que las hace ideales para problemas donde la relación entre las observaciones en diferentes momentos es esencial para hacer predicciones precisas.

Manejo de Secuencias: Las series temporales y otros tipos de datos secuenciales presentan desafíos únicos en el modelado. Las capas LSTM pueden manejar secuencias de longitud variable y aprender relaciones complejas entre las diferentes entradas en diferentes pasos de tiempo.

Memoria a Largo Plazo: A diferencia de las redes neuronales feedforward tradicionales, las LSTM tienen unidades de memoria que les permiten retener información importante a lo largo de la secuencia. Esto permite capturar patrones a largo plazo en los datos, lo que puede ser fundamental para pronósticos precisos en series temporales.

Resistencia al Desvanecimiento del Gradiente: Las capas LSTM están diseñadas para mitigar el problema del desvanecimiento del gradiente en redes neuronales profundas. Esto significa que son más efectivas para entrenar en redes profundas con muchas capas, lo que puede ser útil para capturar relaciones complejas en los datos.

Flexibilidad: Las capas LSTM pueden configurarse con diferentes hiperparámetros, como la cantidad de unidades LSTM en cada capa, el número de capas LSTM, la función de activación y más. Esto permite ajustar el modelo a las características específicas de tus datos y problema.

Pronóstico: Dado que estás interesado en pronosticar la demanda futura basada en datos históricos, un modelo LSTM puede ser particularmente útil, ya que puede capturar patrones y tendencias en las series temporales, permitiéndote hacer predicciones precisas.

La combinación de capas LSTM y otras capas, como capas densas (Dense), en un modelo Sequential de Keras te brinda flexibilidad para diseñar y ajustar el modelo según las necesidades de tu conjunto de datos y el problema específico que estás abordando. En resumen, el uso de capas LSTM en un modelo Keras es adecuado para problemas que involucran secuencias temporales y relaciones a largo plazo entre los datos.


In [9]:
# cargar dataset de ventas 
# este dataset lo utilizaremos como datos de entrenamiento del modelo.
df_train = pd.read_csv("https://media.githubusercontent.com/media/EnriqueEscalante91/Caso-MELI/master/dfventas.csv")
df_train.shape

KeyboardInterrupt: 

In [ ]:
df_train.describe()

In [ ]:
df_train.info()

In [ ]:
# cargar dataset test
# este dataset lo utilizaremos como datos de prueba para comprobar del modelo.
df_test = pd.read_csv("https://media.githubusercontent.com/media/EnriqueEscalante91/Caso-MELI/master/dftest.csv")
df_test.shape

In [ ]:
df_test.describe()

In [ ]:
df_test.info()

# Entregable 2
Graficar una serie de tiempo para una determinada combinación
producto-cliente-agencia.

In [ ]:
#pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Agrupar y sumar las ventas por producto, agencia y cliente
grouped_df = df_train.groupby(['Producto_ID', 'Agencia_ID', 'Cliente_ID'])['Demanda_uni_equil'].sum().reset_index()

# Crear el gráfico de dispersión
plt.figure(figsize=(10, 6))
plt.scatter(grouped_df['Demanda_uni_equil'], grouped_df['Producto_ID'], alpha=0.5)
plt.title('Ventas por Producto, Agencia y Cliente')
plt.xlabel('Ventas')
plt.ylabel('Producto_ID')
plt.tight_layout()
plt.show()
Este código agrupa los datos por producto, agencia y cliente, suma las ventas y luego crea un gráfico de dispersión con las ventas en el eje x y los códigos de producto en el eje y. Cada punto en el gráfico representa una combinación única de producto, agencia y cliente, y el tamaño de los puntos indica el nivel de ventas. Puedes ajustar el tamaño y la apariencia de los puntos según tus preferencias.

In [ ]:
df_test.describe()

In [ ]:
plt.plot